In [1]:
import mne
import datascience as ds
from matplotlib import pyplot as plt
import numpy as np
import os.path as op
import neurods as nds
import sys
%matplotlib inline

# Overview
Our first class is going to cover the background of Electroencephalography (EEG) and Electrocorticography (ECoG). These are the kinds of data we'll use in the first half of this class. Below is a list of topics that we'll focus on.

## Goals for today
* Load some raw data into python
* Simple visualizations to understand the kind of data we have
* Learn some things about the data by scanning the raw values
* Introduction to the MNE library for electrophysiology

---

# Introduction to ECoG and EEG
See the Powerpoint for this week for a quick intro to ECoG and EEG. It has some background on the history of these methods, what kinds of signals they measure, and what kinds of things we'll try to do with them.

# Introduction to MNE-python
Recording from brains generates a particular kind of data. Usually this entails recording some signal at many points in time. This is called a *timeseries*. In neuroscience there are many packages specializing in questions that are of interest to us. One of the first packages that we'll use is called MNE-Python (we'll just call it MNE).

MNE was first created to do one specific thing: compute something called a "Minimum Norm Estimate". This is a technique for determining a source of activity in the brain. However, over time people used for more and more things. Now it is a general framework for analysing electrical signals from the brain.

[Here is a link](http://martinos.org/mne/stable/index.html) to its website and documentation.

# Visualizing electrophysiology data in python

## What is a timeseries?
In neuroscience, the most common form of data is called a *timeseries*. This essentially means that we have a sensor that collects information about some signal in the brain as it changes across time.

A timeseries can be anything from stock market fluctuations, to the changes in score during a basketball game, to electrical fluctuations in the brain.

In this class, we will record two types of timeseries from the human brain:

1. Voltage on the brain surface, recorded from electrodes.
2. A measure of blood flow on the cortex of the brain, recorded with an fMRI machine.

In the first half of this course we'll deal with electrical signals. Today, we'll look at some sample data sets, and start playing around with tools that we have in neuroscience for visualizing and understanding these signals.

## A sample timeseries

In [ ]:
# Here's a dataset that we've recorded, stored in a comma-separated values file
path_data = nds.io.data_list['eeg'] + 'mne_sample/mne_sample_subset.csv'
data = ds.Table.read_table(path_data)

In [ ]:
# First, we'll take a quick look at the data
data.take[0:5]

> * What do each of the columns mean?
> * There is a time column, what do these numbers represent?

In [ ]:
# Now we'll plot a sample timeseries
data.plot('time', ['EEG 001'])

> * What do you notice about this signal? Any repeating patterns?
> * What are the values of the y-axis?

What you see above is electrical activity in the brain. It looks pretty noisy, and a lot of neuroscience analysis entails trying to find signals in the brain that are meaningful and interesting.

In [ ]:
# Let's look at the time values for this signal.
data['time']

These tell us how much time passes between observations of the signal. It's a crucial piece of information in order to do many analyses in neuroscience.

The **sampling frequency** (or sampling rate) is the number of samples we record *per second*. We can calculate this by taking $\frac{1}{time\_between\_samples}$

In [ ]:
# Here's how we calculate the sampling frequency of this signal, what are the units of these numbers?
sampling_freq = 1. / (data['time'][1] - data['time'][0])

In [ ]:
# Let's plot a couple of other channels as well, and see how they compare
data.plot('time', ['EEG 001', 'EEG 013', 'EEG 014'])

> * What do you notice about these channels? How are they different? How are they the same?

## Using MNE to represent our data
Because a timeseries has a specific structure (signals varying over time), there are a number of packages designed to handle this particular kind of data. One of the best python packages for neuroscience is called `MNE-python`. Check out their website here: http://martinos.org/mne/stable/index.html

MNE has code that can handle datasets like the one we've used above. They have a number of "classes" that handle different kinds of datasets. Here is a short-list of classes that you can use:

* `Raw` - raw data sets. (n_channels x n_times)
* `Epochs` - datasets that have been split into windows around times of interest (n_trials x n_channels x n_times_in_window)
* `Evoked` - datasets that have been averaged across trials. (n_channels x n_times_in_window)

Today, we'll focus on the `Raw` data, because it makes viewing your data much easier.

In [ ]:
# We'll load the same dataset, this time with MNE code
path_data = nds.io.data_list['eeg'] + 'mne_sample/mne_sample-raw.fif'
raw = mne.io.Raw(path_data, preload=True)
raw

Let's explore this `Raw` dataset a little bit.

The benefit of using neuro-specific classes (like those in MNE-python) is that we can make assumptions about the data in order to allow us to do more complicated things very easily.

For example, any object that represents data in `MNE-python` will have an `info` attribute. This is a collection of information about the data at hand. It includes information like:

* `info['sfreq']` - The sampling frequency of the data
* `info['ch_names']` - The names of channels in the data
* `info['bads']` - A list of "bad" channels
* `info['chs']` - A more complex collection of channel information, such as xyz location.

There are many other pieces of information related to things that are important in EEG analysis, most of which we won't go into.

In [ ]:
# Here's a list of all the things in an Info object
raw.info

Using MNE objects also gives us **methods** that can do particularly useful things. For example, we can easily crop the data in order to keep only a subset of times:

In [ ]:
# We can easily select subsets of the data for plotting
raw_crop = raw.crop(0, 2)

# However, be careful because this often modifies our data in-place
raw

In [ ]:
# To get around this, we can run `copy` before modifying the data
# Note this is specific to MNE, it may not work on other packages

# Load the data again
raw = mne.io.Raw(nds.io.data_list['eeg'] + 'mne_sample/mne_sample-raw.fif', preload=True)

# Now copy before cropping
raw.copy().crop(0, 2)

We can also use more complex methods for things like data analysis and plotting, which we'll cover later.

You may have noticed that we used a flag `preload=True` when loading this data. By default (or if `preload=False`), MNE will not actually load in the data when you create the object. This is in order to save memory until it is needed. However, our data won't be so large so we'll always preload it.

In [ ]:
# If the data has been preloaded (preload=True), then you can access the
# raw data here:
raw._data

## Plotting raw data with MNE
MNE has a lot of plotting functionality, which is one of the most important things to do in data analysis. Below we'll explore the basic ways in which we can visualize our data.

In [ ]:
# To use this, we'll need to activate "interactive" mode
# This lets us keep plots interactive in the notebook
%matplotlib notebook

First, we'll plot the raw data. This will give us an output of traces, one for each channel. The x-axis is time (in seconds). What we are looking at is the electrical activity (voltage) recorded at the scalp, and generated by neural activity.

In [ ]:
# Putting `_` before the equals sign tells Python to throw away the output
# Otherwise, it will make 2 plots.
_ = raw.plot(scalings='auto')

You can browse this data interactively. Here are a few points:

* Click on points in the data (but not on a black line). A green bar will appear, along with the time (in seconds) for that point.
* Click on a channel, it will turn grey. This is for marking "bad" channels. Bad channel information will be stored in the `Raw` data object.
* Press the arrow keys. Left/Right will move forward and backward in time. up/down will move through channels.
* Press the `-` or `=` key. This will scale the channels up and down in amplitude to make things easier to see.
* If you want to stop the interactivity, press the "power" button to the top right

There is a lot of other functionality in MNE plotting, but we won't go into that here.

> * Why are some lines inverted relative to others?
> * Why are some lines larger in amplitude than others?

## Plotting sensor locations
Patterns of recorded neural activity wouldn't be very useful without knowing where that activity was generated. In EEG and ECoG our sensors sit on the scalp and the surface of the brain, respectively.

MNE has simple functions for plotting the location of sensors with EEG. We can see this below:

In [ ]:
# We can look at the layout of channels on the brain.
# Almost all EEG setups have the same general structure.
_ = raw.plot_sensors(show_names=True)

In [ ]:
# We can also plot in 3d
_ = raw.plot_sensors('3d')

In ECoG things are a bit more complicated, because each ECoG grid is unique (just as each person's brain is unique). We'll cover plotting data in ECoG later on.

# Lab time!